## Defining the linear progam

Decision Variables:
- $x_{i}$ represents allocation of portfolio to asset $i$.

Contraints:
- $\sum_{i} x_i = 1$
- $x_i \ge 0 \forall i$

Objective function:
$$ \max_x \sum_{j} p_j \times (x^T \mu_{j} - \lambda \times x^T \Sigma_j x)$$

Constants being used:
- $\mu_j$, $\Sigma_j$ are expected returns and covariance matrix under scenario $j$
- $p_j$ is the probability of being in scenario $j$
- $\lambda$ is a constant representing risk tolerance.

In [35]:
using JuMP, Gurobi, LinearAlgebra, CSV

In [36]:
m = Model(with_optimizer(Gurobi.Optimizer))

Academic license - for non-commercial use only


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [37]:
num_assets = 4

@variable(m, x[1:num_assets])

@constraint(m, non_neg[i= 1:num_assets], x[i] >= 0)
@constraint(m, sum(x[i] for i=1:num_assets) == 1)

x[1] + x[2] + x[3] + x[4] == 1.0

In [38]:
#Have to load the data.
weak_mu = CSV.read("data/Weak_ExpReturns.csv", header=0)[:, 2]
weak_sigma = convert(Matrix, CSV.read("data/Weak_CovMat.csv", header=1)[:, 2:num_assets+1])

normal_mu = CSV.read("data/Normal_ExpReturns.csv", header=0)[:, 2]
normal_sigma = convert(Matrix, CSV.read("data/Normal_CovMat.csv", header=1)[:, 2:num_assets+1])

strong_mu = CSV.read("data/Strong_ExpReturns.csv", header = 0)[:, 2]
strong_sigma = convert(Matrix, CSV.read("data/Strong_CovMat.csv", header=1)[:, 2:num_assets+1])

4×4 Array{Float64,2}:
  0.190415      0.0071689   -0.000709067  -5.29306e-6
  0.0071689     0.0163268    0.00117289   -5.6025e-6 
 -0.000709067   0.00117289   0.000620389  -3.23118e-6
 -5.29306e-6   -5.6025e-6   -3.23118e-6    6.96783e-7

In [45]:
probs = CSV.read("data/theta0_Probs.csv")[1, 2:num_assets]
normal_prob = probs[1]

strong_prob = probs[2]

weak_prob = probs[3]

0.2797195166414282

In [50]:
transpose(x)*weak_mu

0.10897103662586333 x[1] + 0.08332572891330764 x[2] + 0.0071562978966418596 x[3] + 0.000886494873529021 x[4]

In [52]:
lambda = 1
@objective(m, Max, weak_prob*(transpose(x)*weak_mu - lambda * transpose(x)*weak_sigma*x) + normal_prob*(transpose(x)*normal_mu - lambda * transpose(x)*normal_sigma*x) + strong_prob*(transpose(x)*strong_mu - lambda*transpose(x)*strong_sigma*strong_mu))

-0.22730189831538392 x[1]² - 0.006599678738155798 x[1]*x[2] + 0.0027084544698506583 x[1]*x[3] + 8.886558762003486e-5 x[1]*x[4] - 0.017756857082129925 x[2]² - 0.001411677647950967 x[2]*x[3] - 3.8299078503843204e-5 x[2]*x[4] - 0.0005817531624652348 x[3]² - 4.08963576890834e-6 x[3]*x[4] - 7.161280452480116e-7 x[4]² + 0.16427066588458056 x[1] + 0.07135007431132459 x[2] + 0.0016933976850106282 x[3] + 0.000839691638300929 x[4]

In [54]:
optimize!(m)

Academic license - for non-commercial use only
Optimize a model with 5 rows, 4 columns and 8 nonzeros
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-04, 2e-01]
  QObjective range [1e-06, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 4 columns, 4 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 6.000e+00
 Factor NZ  : 1.000e+01
 Factor Ops : 3.000e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.61182892e+05  2.61642694e+05  3.25e+03 1.64e-01  1.00e+06     0s
   1  -2.33836635e+04  2.37030017e+04  9.39e+01 8.06e-09  4.00e+04     0s
   2   5.03407474e-02  4.52580624e+02  2.59e+00 2.22e-10  1.

In [55]:
value.(x)

4-element Array{Float64,1}:
 0.25546234358510217  
 0.7445376182541931   
 1.9531452766419627e-8
 1.862925200652494e-8 